# Experiment 003: Grid-Based Construction + Optimization

Goal: Generate a NEW initial solution from scratch using grid-based placement, then optimize it to find a DIFFERENT local optimum.

In [1]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely.geometry import Polygon
from shapely import affinity
from shapely.strtree import STRtree
import warnings
warnings.filterwarnings('ignore')

# Set precision for Decimal
getcontext().prec = 25
scale_factor = Decimal('1e15')

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]
TREE_VERTICES = list(zip(TX, TY))

print("Setup complete")

Setup complete


In [2]:
def create_tree_polygon(x, y, deg):
    """Create a tree polygon at position (x, y) with rotation deg"""
    poly = Polygon(TREE_VERTICES)
    poly = affinity.rotate(poly, deg, origin=(0, 0))
    poly = affinity.translate(poly, x, y)
    return poly

def get_bounding_box_side(polygons):
    """Get the side length of the bounding square for a list of polygons"""
    if not polygons:
        return 0
    all_coords = []
    for poly in polygons:
        all_coords.extend(list(poly.exterior.coords))
    xs = [c[0] for c in all_coords]
    ys = [c[1] for c in all_coords]
    return max(max(xs) - min(xs), max(ys) - min(ys))

def has_overlap(polygons):
    """Check if any polygons overlap"""
    if len(polygons) < 2:
        return False
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i:
                if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                    intersection = poly.intersection(polygons[idx])
                    if intersection.area > 1e-10:
                        return True
    return False

print("Helper functions defined")

Helper functions defined


In [3]:
def find_best_grid_trees(n):
    """Find best grid-based arrangement for n trees (from zaburo kernel)"""
    best_score, best_configs = float("inf"), None
    
    for n_even in range(1, n + 1):
        for n_odd in [n_even, n_even - 1]:
            if n_odd < 0:
                continue
            all_configs = []
            rest = n
            r = 0
            while rest > 0:
                m = min(rest, n_even if r % 2 == 0 else n_odd)
                if m <= 0:
                    break
                rest -= m
                
                angle = 0 if r % 2 == 0 else 180
                x_offset = 0 if r % 2 == 0 else 0.7 / 2
                y = r // 2 * 1.0 if r % 2 == 0 else (0.8 + (r - 1) // 2 * 1.0)
                
                for i in range(m):
                    x = 0.7 * i + x_offset
                    all_configs.append((x, y, angle))
                
                r += 1
            
            if len(all_configs) != n:
                continue
                
            # Calculate bounding box
            polygons = [create_tree_polygon(x, y, deg) for x, y, deg in all_configs]
            side = get_bounding_box_side(polygons)
            score = side ** 2
            
            if score < best_score:
                best_score = score
                best_configs = all_configs
    
    return best_score, best_configs

# Test for a few N values
for n in [1, 5, 10, 50, 100, 200]:
    score, configs = find_best_grid_trees(n)
    print(f"N={n:3d}: score={score:.6f}, side={np.sqrt(score):.4f}")

N=  1: score=1.000000, side=1.0000
N=  5: score=4.000000, side=2.0000
N= 10: score=4.840000, side=2.2000


N= 50: score=24.010000, side=4.9000


N=100: score=39.690000, side=6.3000


N=200: score=81.000000, side=9.0000


In [ ]:
# Generate grid-based solution for all N=1 to 200
print("Generating grid-based solutions for N=1 to 200...")

grid_solutions = {}
total_score = 0

for n in range(1, 201):
    score, configs = find_best_grid_trees(n)
    grid_solutions[n] = configs
    total_score += score / n
    if n % 50 == 0:
        print(f"N={n}: cumulative score = {total_score:.6f}")

print(f"\nTotal grid-based score: {total_score:.6f}")
print(f"Target score: 68.919154")
print(f"Pre-optimized score: 70.676102")

In [ ]:
# Create submission CSV from grid-based solution
def format_s_value(v):
    return f's{v}'

rows = []
for n in range(1, 201):
    configs = grid_solutions[n]
    for i, (x, y, deg) in enumerate(configs):
        rows.append({
            'id': f'{n:03d}_{i}',
            'x': format_s_value(x),
            'y': format_s_value(y),
            'deg': format_s_value(deg)
        })

grid_df = pd.DataFrame(rows)
print(f"Created {len(grid_df)} rows")
print(grid_df.head(10))

In [ ]:
# Save grid-based solution
grid_df.to_csv('/home/code/experiments/003_grid_based_construction/grid_initial.csv', index=False)
print("Saved grid_initial.csv")

# Also save to working directory for optimizer
grid_df.to_csv('/home/code/experiments/003_grid_based_construction/submission.csv', index=False)
print("Saved submission.csv for optimizer")

In [ ]:
# Validate no overlaps in grid solution
print("Validating grid solution for overlaps...")
overlap_count = 0

for n in range(1, 201):
    configs = grid_solutions[n]
    polygons = [create_tree_polygon(x, y, deg) for x, y, deg in configs]
    if has_overlap(polygons):
        overlap_count += 1
        print(f"N={n}: OVERLAP DETECTED!")

if overlap_count == 0:
    print("Grid solution has NO OVERLAPS \u2713")
else:
    print(f"WARNING: {overlap_count} configurations have overlaps!")

In [ ]:
# Now run the C++ optimizer on this new starting point
import subprocess
import os

os.chdir('/home/code/experiments/003_grid_based_construction')

# Copy the optimizer
if not os.path.exists('bbox3_compiled'):
    subprocess.run(['cp', '/home/code/experiments/002_bbox3_optimization/bbox3_compiled', '.'])
    subprocess.run(['chmod', '+x', 'bbox3_compiled'])

print("Running bbox3 optimizer on grid-based initial solution...")
result = subprocess.run(
    ['timeout', '300', './bbox3_compiled', '-n', '2000', '-r', '30'],
    capture_output=True, text=True
)
print(result.stdout[-3000:] if len(result.stdout) > 3000 else result.stdout)
if result.stderr:
    print("STDERR:", result.stderr[-500:])

In [ ]:
# Check the optimized result
optimized_df = pd.read_csv('submission.csv')
print(f"Optimized solution has {len(optimized_df)} rows")

# Calculate score
def parse_s_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

optimized_df['x_val'] = optimized_df['x'].apply(parse_s_value)
optimized_df['y_val'] = optimized_df['y'].apply(parse_s_value)
optimized_df['deg_val'] = optimized_df['deg'].apply(parse_s_value)

total_score = 0
for n in range(1, 201):
    prefix = f'{n:03d}_'
    group = optimized_df[optimized_df['id'].str.startswith(prefix)]
    polygons = [create_tree_polygon(row['x_val'], row['y_val'], row['deg_val']) for _, row in group.iterrows()]
    side = get_bounding_box_side(polygons)
    total_score += side**2 / n

print(f"\nOptimized grid-based score: {total_score:.6f}")
print(f"Pre-optimized baseline: 70.676102")
print(f"Target: 68.919154")

In [ ]:
# Compare with baseline and decide which to use
baseline_score = 70.676102
if total_score < baseline_score:
    print(f"\n*** IMPROVEMENT! Grid-based approach is better by {baseline_score - total_score:.6f} ***")
    # Copy to submission
    optimized_df[['id', 'x', 'y', 'deg']].to_csv('/home/submission/submission.csv', index=False)
    print("Saved to /home/submission/submission.csv")
else:
    print(f"\nGrid-based approach is worse by {total_score - baseline_score:.6f}")
    print("Keeping baseline solution")
    # Copy baseline to submission
    import shutil
    shutil.copy('/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/ensemble.csv', '/home/submission/submission.csv')
    print("Copied baseline to /home/submission/submission.csv")

In [ ]:
# Summary
print("="*50)
print("EXPERIMENT 003 SUMMARY")
print("="*50)
print(f"Grid-based initial score: {sum(grid_solutions[n][0]**2 if isinstance(grid_solutions[n], tuple) else 0 for n in range(1, 201)):.6f}")
print(f"Grid-based optimized score: {total_score:.6f}")
print(f"Pre-optimized baseline: {baseline_score:.6f}")
print(f"Target: 68.919154")
print(f"Best score: {min(total_score, baseline_score):.6f}")
print("="*50)